In [52]:
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [2]:
test_data = pd.read_csv('test_data.csv')
train_data = pd.read_csv('train_data.csv')
train_labels = pd.read_csv('train_labels.csv')

In [3]:
del train_data['is_duplicate']

In [4]:
train_data_labels = pd.merge(train_data,train_labels, how='left')

In [5]:
train_data_labels = train_data_labels.dropna()

In [44]:
train_data_labels[train_data_labels['is_duplicate']==1].head(20)

,id,question1,question2,is_duplicate
5,5,"[astrology, i, capricorn, sun, cap, moon, cap,...","[im, triple, capricorn, sun, moon, ascendant, ...",1
7,7,"[how, i, good, geologist]","[what, i, great, geologist]",1
11,11,"[how, i, read, find, youtube, comments]","[how, i, see, youtube, comments]",1
12,12,"[what, make, physics, easy, learn]","[how, make, physics, easy, learn]",1
13,13,"[what, first, sexual, experience, like]","[what, first, sexual, experience]",1
15,16,"[what, manipulation, mean]","[what, manipulation, means]",1
17,18,"[why, many, quora, users, posting, questions, ...","[why, people, ask, quora, questions, answered,...",1
25,29,"[how, i, prepare, ca, final, law]","[how, one, know, heshe, completely, prepare, c...",1
27,31,"[what, special, cares, someone, nose, gets, st...","[how, i, keep, nose, getting, stuffy, night]",1
28,32,"[what, game, thrones, villain, would, likely, ...","[what, game, thrones, villain, would, like, me...",1


In [7]:
train_data_labels['question1'] = train_data_labels['question1'].str.replace('[^\w\s]','')
train_data_labels['question2']= train_data_labels['question2'].str.replace('[^\w\s]','')

In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [9]:
train_data_labels['question1'] = train_data_labels['question1'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
train_data_labels['question2'] = train_data_labels['question2'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [11]:
train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [13]:
from textblob import TextBlob
#train_data_labels['question1'] = train_data_labels['question1'].apply(lambda x: str(TextBlob(x).correct())
#train_data_labels['question2'] = train_data_labels['question2'].apply(lambda x: str(TextBlob(x).correct())                                                                     

In [29]:
train_data_labels['question1'].apply(lambda x : TextBlob(x).words)

0         [what, step, step, guide, invest, share, marke...
1                [what, story, kohinoor, kohinoor, diamond]
2         [how, i, increase, speed, internet, connection...
3                 [why, i, mentally, lonely, how, i, solve]
4         [which, one, dissolve, water, quikly, sugar, s...
5         [astrology, i, capricorn, sun, cap, moon, cap,...
6                                   [should, i, buy, tiago]
7                                 [how, i, good, geologist]
8                                [when, use, シ, instead, し]
9         [motorola, company, can, i, hack, charter, mot...
10        [method, find, separation, slits, using, fresn...
11                  [how, i, read, find, youtube, comments]
12                       [what, make, physics, easy, learn]
13                  [what, first, sexual, experience, like]
14        [what, laws, change, status, student, visa, gr...
15                               [what, manipulation, mean]
16                 [why, girls, want, fr

In [23]:
TextBlob(train_data_labels['question1'][8]).words

WordList(['when', 'use', 'シ', 'instead', 'し'])

In [30]:
from nltk.tokenize import word_tokenize
import nltk

#train_data_labels.apply(lambda row: word_tokenize(row['question1']), axis=1)
train_data_labels['question1'] = train_data_labels['question1'].apply(word_tokenize)
train_data_labels['question2'] = train_data_labels['question2'].apply(word_tokenize)

In [31]:
train_data_labels.head()

,id,question1,question2,is_duplicate
0,0,"[what, step, step, guide, invest, share, marke...","[what, step, step, guide, invest, share, market]",0
1,1,"[what, story, kohinoor, kohinoor, diamond]","[what, would, happen, indian, government, stol...",0
2,2,"[how, i, increase, speed, internet, connection...","[how, internet, speed, increased, hacking, dns]",0
3,3,"[why, i, mentally, lonely, how, i, solve]","[find, remainder, math2324math, divided, 2423]",0
4,4,"[which, one, dissolve, water, quikly, sugar, s...","[which, fish, would, survive, salt, water]",0


In [32]:
question1 = train_data_labels['question1'].tolist()

In [33]:
question2 = train_data_labels['question2'].tolist()

In [36]:
questions = question1 + question2

In [37]:
questions

[['what', 'step', 'step', 'guide', 'invest', 'share', 'market', 'india'],
 ['what', 'story', 'kohinoor', 'kohinoor', 'diamond'],
 ['how', 'i', 'increase', 'speed', 'internet', 'connection', 'using', 'vpn'],
 ['why', 'i', 'mentally', 'lonely', 'how', 'i', 'solve'],
 ['which',
  'one',
  'dissolve',
  'water',
  'quikly',
  'sugar',
  'salt',
  'methane',
  'carbon',
  'di',
  'oxide'],
 ['astrology',
  'i',
  'capricorn',
  'sun',
  'cap',
  'moon',
  'cap',
  'risingwhat',
  'say'],
 ['should', 'i', 'buy', 'tiago'],
 ['how', 'i', 'good', 'geologist'],
 ['when', 'use', 'シ', 'instead', 'し'],
 ['motorola',
  'company',
  'can',
  'i',
  'hack',
  'charter',
  'motorolla',
  'dcx3400'],
 ['method', 'find', 'separation', 'slits', 'using', 'fresnel', 'biprism'],
 ['how', 'i', 'read', 'find', 'youtube', 'comments'],
 ['what', 'make', 'physics', 'easy', 'learn'],
 ['what', 'first', 'sexual', 'experience', 'like'],
 ['what',
  'laws',
  'change',
  'status',
  'student',
  'visa',
  'green',
  

In [39]:
import gensim 

In [40]:
model = gensim.models.Word2Vec(questions, min_count=1,size=300,workers=4)

In [46]:
print(model.similarity('radio', 'channels'))

0.42680474515911065


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
